<a href="https://colab.research.google.com/github/njho/Notebooks/blob/master/PyTorch_60_Minute_Tutorial_That_isn't_60_Minutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        #1 input image channel, 6, 5x5 square convolution
        #kernel
#         (1 x 32 x 32) -> (6 x 32 x 32)
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        

In [0]:
def outputSize(in_size, kernel_size, stride, padding):

    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1

    return(output)

In [6]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
params = list(net.parameters())
print(len(params))
for i in range(len(params)):
    print(params[i].size())  # conv1's .weight
    
print(params[1])

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])
Parameter containing:
tensor([ 0.1127, -0.0683,  0.0210,  0.0794,  0.1614,  0.0990], requires_grad=True)


In [8]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0752, -0.0298, -0.0476, -0.1167, -0.0540,  0.0455, -0.0601, -0.0883,
          0.1051, -0.0971]], grad_fn=<ThAddmmBackward>)


### I missed this originally

In [0]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [10]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6247, grad_fn=<MseLossBackward>)


In [11]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### A Way of Printing the Full Graph

In [12]:
def print_graph(g, level=0):
    if g == None: return
    print('*'*level*4, g)
    for subg in g.next_functions:
        print_graph(subg[0], level+1)

print_graph(loss.grad_fn, 0)

**** <ThAddmmBackward object at 0x7f4ee01bfda0>
******** <ExpandBackward object at 0x7f4ee01bfe48>
************ <AccumulateGrad object at 0x7f4ee01bff28>
******** <ReluBackward object at 0x7f4ee01bfe80>
************ <ThAddmmBackward object at 0x7f4ee01bff28>
**************** <ExpandBackward object at 0x7f4ee01bff98>
******************** <AccumulateGrad object at 0x7f4ee01cd0b8>
**************** <ReluBackward object at 0x7f4ee01bffd0>
******************** <ThAddmmBackward object at 0x7f4ee01cd160>
************************ <ExpandBackward object at 0x7f4ee01cd0f0>
**************************** <AccumulateGrad object at 0x7f4ee01cd2e8>
************************ <ViewBackward object at 0x7f4ee01cd0b8>
**************************** <MaxPool2DWithIndicesBackward object at 0x7f4ee01cd4a8>
******************************** <ReluBackward object at 0x7f4ee01b80f0>
************************************ <ThnnConv2DBackward object at 0x7f4ee01b8400>
**************************************** <MaxPool2DWit

### Back on Track

In [13]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0059,  0.0019,  0.0014,  0.0088, -0.0003, -0.0007])


In [0]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [15]:
print(net.parameters())

<generator object Module.parameters at 0x7f4ee020deb8>
